In [1]:
from pyspark import SparkContext
import numpy as np
import math

sc = SparkContext()

# Ejercicio1 

In [2]:
rdd_alt = sc.textFile('alturas_v0.csv')

alturas = (rdd_alt
          .map(lambda x: float(x)) # Transformamos de string a float
          .filter(lambda x: x > 0) # Eliminamos los negativos que ponen -100
          .map(lambda x: x*100 if x < 3 else x) # Si el valor es menor que 3 lo multiplicamos por 100 (Uso 3 por si hay gente que mida 2 metros en la lista)
          .map(lambda x: np.array([x, x*x, 1])) # Guardamos el valor normal, su cuadrado y el valor 1 que usaremos para contar cuántos valores tenemos
          .reduce(lambda x,y: x + y))    # Como es array podemos hacer esto que numpy hace que se sume correctamente

# Para calcular la media simplemente dividimos el primer valor que contiene la suma de todos y divimos entre el último que tiene el número total de datos
media = alturas[0] / alturas [2]

# Para la desviación típica cogemos la suma de cuadrados dividida por el número de datos, le restamos la media al cuadrado y hacemos la raiz de todo
std = math.sqrt((alturas[1]/alturas[2]) - media**2)

print('Media: {:.3f}'.format(media),', Desviación típica {:.3f}'.format(std))

Media: 168.938 , Desviación típica 7.834


# Ejercicio 2

In [3]:
rdd_alt = sc.textFile('alturas.csv')

# Hacemos primero una trasnformación igual a la anterior pero teniendo en cuenta que ahora cada línea tiene dos elementos uno para el género y otro para la altura
alturas = (rdd_alt
          .map(lambda x: x.split(','))
          .map(lambda x: [x[0],float(x[1])])
          .filter(lambda x: x[1]>0)
          .map(lambda x: [x[0], x[1]*100 if x[1] < 3 else x[1]]))

# Definimos mejor una función para el procesado que aplicaremos 2 veces. Hacemos la misma transformación del ejercicio anterior solo que ahora primero filtramos por género
def media_std(rdd,genero):
    rdd_gen = (rdd.filter(lambda x: x[0]==genero)
               .map(lambda x: np.array([x[1],x[1]*x[1],1]))
               .reduce(lambda x,y: x+y))
    
    media = rdd_gen[0] / rdd_gen[2]
    std = math.sqrt((rdd_gen[1]/rdd_gen[2]) - media**2)
    
    return media, std

media_hombres, std_hombres = media_std(alturas, 'H')
media_mujeres, std_mujeres = media_std(alturas, 'M')

print('Media hombres: {:.3f}'.format(media_hombres),', Desviación típica hombres: {:.3f}'.format(std_hombres))
print('Media mujeres: {:.3f}'.format(media_mujeres),', Desviación típica mujeres: {:.3f}'.format(std_mujeres))

Media hombres: 173.461 , Desviación típica hombres: 7.531
Media mujeres: 164.168 , Desviación típica mujeres: 4.685
